In [1]:
import numpy as np
import polars as pl

In [2]:
from pathlib import Path


dpath = Path('../../dataset')
emb_dir = dpath / 'embeddings'
dtype = 'small'
articles = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/articles.parquet')

behaviors_train = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/train/behaviors.parquet')
history_train = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/train/history.parquet')

behaviors_val = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/validation/behaviors.parquet')
history_val = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/validation/history.parquet')

In [3]:
embeddings = pl.read_parquet(emb_dir / 'google_bert_base_multilingual_cased' / 'bert_base_multilingual_cased.parquet').sort('article_id')[:50000]
embeddings.columns = ['article_id', 'embedding']
embeddings = embeddings.with_row_index()
embeddings.head(2)

index,article_id,embedding
u32,i32,list[f32]
0,3000022,"[-0.350606, 0.003437, … 0.001947]"
1,3000063,"[-0.003448, 0.227659, … -0.057015]"


In [4]:
all_zero_embeddings = embeddings.with_columns(pl.col('embedding').list.eval(pl.element() == 0.0).list.all().alias('check'))
are_all_zero_embeddings_present = len(all_zero_embeddings.filter(pl.col('check') == True)) > 0
are_all_zero_embeddings_present

False

In [17]:
m = np.array([np.array(x) for x in embeddings['embedding'].to_numpy()])
row_norms = np.linalg.norm(m, axis=1, keepdims=True)
m_norm = m / row_norms
m.shape

(50000, 768)

In [6]:
# row_norms = np.linalg.norm(m, axis=1, keepdims=True)
# shrink_term = 1e-5 if are_all_zero_embeddings_present else 0.
# m_norm = m / (row_norms + shrink_term)

# sim_matrix = np.dot(m_norm, m_norm.T)
# sim_matrix.shape

In [7]:
# from RecSys_Course_AT_PoliMi.Recommenders.KNN.UserKNNCFRecommender import Compute_Similarity
# from scipy import sparse as sps
# sim = Compute_Similarity(sps.csr_matrix(m).T, topK=100, normalize=True, similarity='cosine')
# m_sim_sparse = sim.compute_similarity()


# Build feature

In [8]:
article_emb_mapping = embeddings.select('index', 'article_id')
article_emb_mapping

index,article_id
u32,i32
0,3000022
1,3000063
2,3000613
3,3000700
4,3000840
…,…
49995,7090834
49996,7090903
49997,7091059


In [9]:
artilce_ids = article_emb_mapping['article_id'].unique().to_numpy()
artilce_ids

array([3000022, 3000063, 3000613, ..., 7091059, 7091113, 7091148],
      dtype=int32)

In [10]:
import torch
import numpy as np

# Example NumPy matrix (replace this with your actual data)
numpy_matrix = m

# Convert to PyTorch tensor
tensor_matrix = torch.tensor(numpy_matrix, dtype=torch.float32)

# Normalize the tensor along axis 1 (rows)
tensor_matrix_normalized = torch.nn.functional.normalize(tensor_matrix, p=2, dim=1)

# Compute the similarity matrix using matrix multiplication
similarity_matrix = torch.matmul(tensor_matrix_normalized, tensor_matrix_normalized.T)

print("Similarity Matrix:")
print(similarity_matrix)


Similarity Matrix:
tensor([[1.0000, 0.9066, 0.8908,  ..., 0.9053, 0.9019, 0.9083],
        [0.9066, 1.0000, 0.8627,  ..., 0.9912, 0.9887, 0.9890],
        [0.8908, 0.8627, 1.0000,  ..., 0.8633, 0.8643, 0.8767],
        ...,
        [0.9053, 0.9912, 0.8633,  ..., 1.0000, 0.9921, 0.9888],
        [0.9019, 0.9887, 0.8643,  ..., 0.9921, 1.0000, 0.9884],
        [0.9083, 0.9890, 0.8767,  ..., 0.9888, 0.9884, 1.0000]])


In [16]:
import random
df = behaviors_train.select('impression_id', 'user_id', pl.col('article_ids_inview').alias('article'))\
    .join(history_train, on='user_id')\
    .with_columns(
        pl.Series([random.sample(artilce_ids, 3)] * len(df))
    )

df.head(2)

TypeError: Population must be a sequence.  For dicts or sets, use sorted(d).

In [15]:
np.random(1)

TypeError: 'module' object is not callable